In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
import torch.utils.data.dataloader as Data
import os
import time
from torch.autograd import Variable
from DAST_utils import *
from DAST_Network import *
from torch.utils.data import TensorDataset,DataLoader
import sys
from tqdm import tqdm

In [50]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [51]:
#Myscore function
def myScore(Target, Pred):
    tmp1 = 0
    tmp2 = 0
    for i in range(len(Target)):
        if Target[i] > Pred[i]:
            tmp1 = tmp1 + math.exp((-Pred[i] + Target[i]) / 13.0) - 1
        else:
            tmp2 = tmp2 + math.exp((Pred[i] - Target[i]) / 10.0) - 1
    tmp = tmp1 + tmp2
    return tmp

In [52]:
prediction_loss = []
# Load preprocessed data
X_train = sio.loadmat('./mill_window_size_trainX.mat')  # load sliding window preprocessed and Statistical features processed data (mean value and regression coefficient estimates feature)
X_train = X_train['train1X']

# X_train = X_train.reshape(len(X_train),42,14)
Y_train = sio.loadmat('./mill_window_size_trainY.mat')
Y_train = Y_train['train1Y']

In [53]:
print(type(X_train))
print(type(Y_train))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [54]:
print(Y_train.shape)

(10020, 1)


In [55]:
X_train = Variable(torch.Tensor(X_train).float())
Y_train = Variable(torch.Tensor(Y_train).float())

In [56]:
#Hyperparameters
batch_size = 256
dim_val = 32
dim_attn = 32
dim_val_t = 32
dim_attn_t = 32
dim_val_s = 32
dim_attn_s = 32
n_heads = 4
n_decoder_layers = 1
n_encoder_layers = 2
max_rul = 125
lr = 0.001
epochs = 150
time_step = 40
dec_seq_len = 4
output_sequence_length = 1
input_size = 6

In [57]:
#Dataloader
train_dataset = TensorDataset(X_train,Y_train)
train_loader = Data.DataLoader(dataset=train_dataset,batch_size = batch_size,shuffle=True)

In [58]:
# Initialize model parameters
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# debug = True
debug = False
model = DAST(dim_val_s,dim_attn_s,dim_val_t,dim_attn_t,dim_val, dim_attn,time_step,input_size,dec_seq_len,output_sequence_length, n_decoder_layers, n_encoder_layers, n_heads, debug)
# model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

In [59]:
print(next(model.parameters()).device)

cpu


In [60]:
#Training  and testing
loss_list = []
train_loss_list = []
test_loss_list = []
train_time = []
test_time = []
model_loss = 1000

In [61]:
for epoch in range(epochs):
    #training
    model.train()
    start1 = time.time()
    loop = tqdm(train_loader, leave=True)
    for i,(X, Y) in enumerate(loop):
        # batch_X = X.to(device)
        # batch_Y = Y.to(device)
        out = model(X)
        loss = torch.sqrt(criterion(out*max_rul, Y*max_rul))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    end1 = time.time()
    train_time.append(end1 - start1)
    loss_eopch = np.mean(np.array(loss_list))
    train_loss_list.append(loss_eopch)
    print('epoch = ',epoch,
            'train_loss = ',loss_eopch.item())


100%|██████████| 40/40 [00:03<00:00, 10.04it/s]


epoch =  0 train_loss =  7415.392248535156


100%|██████████| 40/40 [00:03<00:00, 10.10it/s]


epoch =  1 train_loss =  7173.698083496094


100%|██████████| 40/40 [00:03<00:00, 10.12it/s]


epoch =  2 train_loss =  6922.416552734375


100%|██████████| 40/40 [00:04<00:00,  9.80it/s]


epoch =  3 train_loss =  6672.367248535156


  8%|▊         | 3/40 [00:00<00:04,  9.13it/s]


KeyboardInterrupt: 